In [2]:
!pip install huggingface_hub sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 10.4 MB/s eta 0:00:00


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `Arpita` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate w

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define model name
model_name = "microsoft/biogpt"

# Load and save the model and tokenizer
biogpt_tokenizer = AutoTokenizer.from_pretrained(model_name)
biogpt_model = AutoModelForCausalLM.from_pretrained(model_name)

# Define model name
model_name = "dmis-lab/biobert-base-cased-v1.2"

# Load and save the model and tokenizer
biobert_tokenizer = AutoTokenizer.from_pretrained(model_name)
biobert_model = AutoModelForCausalLM.from_pretrained(model_name)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [5]:
from transformers import AutoModel, AutoConfig
import torch
from pathlib import Path

def save_huggingface_model(model, save_dir):
    """Save Hugging Face model to a directory."""
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    model.save_pretrained(save_dir)
    print(f"Merged model saved at {save_dir}")

def ties_merge_models(model_a, model_b, threshold=1e-5):
    """
    Perform TIES merging on two Hugging Face models.

    Args:
        model_a: Model A (e.g., BioGPT).
        model_b: Model B (e.g., BioBERT).
        threshold: Threshold to consider marginal parameter changes.
    Returns:
        A merged Hugging Face model.
    """
    state_dict_a = model_a.state_dict()
    state_dict_b = model_b.state_dict()
    merged_state_dict = {}

    for key in state_dict_a.keys():
        if key in state_dict_b:
            param_a = state_dict_a[key]
            param_b = state_dict_b[key]

            # Reset marginal changes
            if torch.abs(param_a - param_b).mean() < threshold:
                merged_state_dict[key] = param_a
                continue

            # Resolve sign conflicts
            sign_mask = torch.sign(param_a) == torch.sign(param_b)
            resolved_param = torch.where(sign_mask, (param_a + param_b) / 2, param_a.abs().max(param_b.abs()))

            # Selective merging based on resolved sign
            merged_state_dict[key] = resolved_param
        else:
            merged_state_dict[key] = state_dict_a[key]  # Default to model A's parameters

    # Load the merged state dict into model A's architecture
    model_a.load_state_dict(merged_state_dict)
    return model_a


In [6]:
from transformers import AutoTokenizer
from pathlib import Path

def save_merged_tokenizer(tokenizer, save_dir):
    """Save Hugging Face tokenizer to a directory."""
    # Save the updated tokenizer
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    tokenizer.save_pretrained(save_dir)
    print(f"Merged tokenizer saved at {save_dir}")

def merge_tokenizer_vocabularies(tokenizer_a, tokenizer_b, save_dir):
    """
    Merge the vocabularies of two tokenizers.

    Args:
        tokenizer_a: first tokenizer
        tokenizer_b: second tokenizer
        save_dir: Path to save the merged tokenizer.
    """

    # Extract vocabularies
    vocab_a = set(tokenizer_a.get_vocab().keys())
    vocab_b = set(tokenizer_b.get_vocab().keys())

    # Merge vocabularies
    merged_vocab = vocab_a.union(vocab_b)

    # Update tokenizer A's vocabulary (base tokenizer)
    current_vocab = tokenizer_a.get_vocab()
    added_tokens = [token for token in merged_vocab if token not in current_vocab]
    tokenizer_a.add_tokens(added_tokens)

    return tokenizer_a

save_dir = "/merged_model"

# Perform TIES merging
merged_model = ties_merge_models(biogpt_model, biobert_model)

# Save the merged model
save_huggingface_model(merged_model, save_dir)

# Perfrom tokenizer merging
merged_tokenizer = merge_tokenizer_vocabularies(biogpt_tokenizer, biobert_tokenizer, save_dir)

# save merged tokenizer
save_merged_tokenizer(merged_tokenizer, save_dir)


Merged model saved at /merged_model
Merged tokenizer saved at /merged_model


In [12]:
# Define a prompt for medical text generation
prompt = "Describe the pathophysiology of diabetes mellitus type 2."

# Encode the prompt
inputs = biobert_tokenizer(prompt, return_tensors="pt")

# Generate text
outputs = merged_model.generate(inputs["input_ids"], max_length=200, num_return_sequences=1)

# Decode and print the generated text
generated_text = merged_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

cancer 6.8 moderate transduction alized diameters hemobioactive linear edible half platinum anaerobic 71 RESULTS use no significant differences in the mean values of the parameters of the study.
